# **Credit Card Fraud Prediction**

In this notebook, I use an ensemble classifier to predict whether credit card transactions are fraud or not. A very small portion of credit card transactions are fraudulent so one of the biggest problems in this scenario is sampling for such a heavily imbalanced dataset. In the model below I use a bagging classifier to train many estimators based on a balanced subset of credit card transaction data to ensure that both fraudulent and legitimate transactions get equal representation when training the model. 

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Below, I load the credit card dataset provided by Kaggle. The features provided are heavily masked and scaled to protect the user's identities and transaction histories. This limits the intuitiveness of the dataset for feature selection but we are still able to observe the correlation coefficients between 'Class', where 1 represents a fraudulent transaction and 0 represents legitimate transaction. 

In [ ]:
credit_df = pd.read_csv('creditcard.csv')
credit_df.corr()['Class']

Time     -0.012323
V1       -0.101347
V2        0.091289
V3       -0.192961
V4        0.133447
V5       -0.094974
V6       -0.043643
V7       -0.187257
V8        0.019875
V9       -0.097733
V10      -0.216883
V11       0.154876
V12      -0.260593
V13      -0.004570
V14      -0.302544
V15      -0.004223
V16      -0.196539
V17      -0.326481
V18      -0.111485
V19       0.034783
V20       0.020090
V21       0.040413
V22       0.000805
V23      -0.002685
V24      -0.007221
V25       0.003308
V26       0.004455
V27       0.017580
V28       0.009536
Amount    0.005632
Class     1.000000
Name: Class, dtype: float64

After observing the feature correlations, we select the below subset and split the fradulent and valid transaction subsets in half for training and testing. There are about 400 fradulent and 300000 valid transactions so our training subsets will be drawn from the first half while our testing subsets will be drawn from the second half. We also leverage a MinMaxScaler to normalize our data.

In [ ]:
selected_features = ['V1', 'V2', 'V3', 'V4', 'V5', 'V7', 'V9', 'V10', 'V11', 'V12', 'V14', 'V16', 'V17', 'V18']

fraud_split_index = 200
valid_split_index = 150000

for col in credit_df.select_dtypes(include=['object']).columns:
    credit_df[col] = credit_df[col].fillna(value=0)
    credit_df[col] = pd.Categorical(credit_df[col], categories=credit_df[col].unique()).codes

fraud_training_indexes = credit_df[credit_df['Class'] == 1].index[:fraud_split_index]
valid_training_indexes = credit_df[credit_df['Class'] == 0].index[:valid_split_index]

fraud_test_indexes = credit_df[credit_df['Class'] == 1].index[fraud_split_index:]
valid_test_indexes = credit_df[credit_df['Class'] == 0].index[valid_split_index:]

min_max_scaler = preprocessing.MinMaxScaler()
X = pd.DataFrame(min_max_scaler.fit_transform(credit_df), columns=credit_df.columns, index=credit_df.index).drop('Class', axis=1)[selected_features]
y = credit_df['Class']


For more equal representation of both classes while training our model, we select 10 different subsets of 100 and 50 elements respectively for valid and fraud classes to fit an AdaBoostClassifier. I choose this 2:1 sampling ratio so our model doesn't overfit the fraudulent class. Leveraging an ensemble classifier allows us to train many different classifiers and take a final vote to determine the best answer.

In [ ]:
clf = BaggingClassifier(base_estimator=AdaBoostClassifier(), n_estimators=100, random_state=0)

for i in range(10):
    random_fraud_index = np.random.choice(fraud_training_indexes, 50)
    random_valid_index = np.random.choice(valid_training_indexes, 100)
    train_subset = np.concatenate((random_fraud_index.astype(int), random_valid_index.astype(int)), axis=0)
    clf.fit(X.iloc[train_subset.astype(int)], y.iloc[train_subset.astype(int)])


Assessing performance of our classifier on such a heavily imbalanced dataset poses a great challenge. If we were to train a naive classifier that predicts class 0 for every sample in our dataset we would get a classifier with over 99.8% accuracy but this does nothing to solve the problem at hand because we won't catch any fraud transactions. Here, we need to take a deeper dive into the confusion matrix and assess our precision and recall scores to see what mistakes our model is making. We examine the entire second half of the credit card dataset, the half that our model hasn't seen, to understand how our model performs at a large scale. While we trained it on a relatively even subset of both classes our model will be applied on very heavily imbalanced data so we must see how it works in practice.

In [ ]:
test_subset = np.concatenate((fraud_test_indexes, valid_test_indexes), axis=0)
X_test = X.iloc[test_subset.astype(int)]
pred = clf.predict(X_test)

print(confusion_matrix(pred,y.iloc[test_subset]))
print(classification_report(pred, y.iloc[test_subset]))


[[133526     49]
 [   789    243]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    133575
           1       0.83      0.24      0.37      1032

    accuracy                           0.99    134607
   macro avg       0.91      0.62      0.68    134607
weighted avg       0.99      0.99      0.99    134607



In our results above, we see that we have recall and precision scores of .24 and .83 respectively. We are catching over 83% of fraud transactions while only falsely flagging 0.58% of valid transactions as false. Let's explore these numbers on an XGBoost.

In [ ]:
clf = xgb.XGBClassifier(random_state=0)
for i in range(10):
    random_fraud_index = np.random.choice(fraud_training_indexes, 50)
    random_valid_index = np.random.choice(valid_training_indexes, 100)
    train_subset = np.concatenate((random_fraud_index.astype(int), random_valid_index.astype(int)), axis=0)
    clf.fit(X.iloc[train_subset.astype(int)], y.iloc[train_subset.astype(int)])
    
test_subset = np.concatenate((fraud_test_indexes, valid_test_indexes), axis=0)
X_test = X.iloc[test_subset.astype(int)]
pred = clf.predict(X_test)

print(confusion_matrix(pred,y.iloc[test_subset]))
print(classification_report(pred, y.iloc[test_subset]))


[[129893     39]
 [  4422    253]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98    129932
           1       0.87      0.05      0.10      4675

    accuracy                           0.97    134607
   macro avg       0.92      0.53      0.54    134607
weighted avg       0.96      0.97      0.95    134607



Here, we get precision and recall of 0.87 and 0.05. We are correctly flagging 87% of fraud transactions, a notable improvement, but we are now flagging 3.3% of valid transactions as fraud. There is no clear answer over which model performs better, the former has better recall while the latter has better precision. In this particular instance I prefer the former, I don't think the 4% improvement in detecting fraud outweighs a 5 time increase in valid transactions being denied. 